In [ ]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [1]:
import warnings
warnings.filterwarnings('ignore')
import codecs
import base64
import hashlib
import ipywidgets as widgets
import pandas as pd
import time
from os.path import isfile, join
from IPython.display import HTML, display, clear_output

In [10]:
output = widgets.Output()
df = pd.read_excel('concept_comparison_maxgraph_missing.xlsx')
selected_rows = df[df['parent'].str.contains('**', regex=False)]
selected_rows.sort_values('max_frequency', ascending=False, inplace=True)
selected_rows.reset_index(drop=True, inplace=True)
list1 = selected_rows['child'].str.replace('.dot', '.png')
list2 = selected_rows['parent'].str.split().str.get(0).str.cat(['_'] * selected_rows.shape[0]).str.cat(selected_rows['child'].str.split().str.get(0)).str.cat(['.png'] * selected_rows.shape[0])
list2_title = selected_rows['parent'].str.replace('.dot', '.png').str.replace(' **', '', regex=False)
model_folder = 'graph_visualization'
missing_folder = 'missing_lexemes'
counter = 0

In [ ]:
green_box_layout = widgets.Layout()
green_box_layout.width = '50 px'
green_box_layout.height = '100 px'
green_box_layout.border = '2px solid'

res = ''

def on_button_next_clicked(event):
    global counter
    global res
    res += list2[counter].split('_')[0] + '\t' + list2[counter].split('_')[1].replace('.png', '') \
    + '\t' + str(validation_choice.value) + '\t' + comment.value + '\n'
    b64 = base64.b64encode(res.encode())
    payload = b64.decode()
    link_download.value = "<a download=responses.txt href=\"data:text/csv;base64," + payload + "\" download>Responses.txt</a>"
    counter += 1
    if counter == selected_rows.shape[0]:
        validation_choice.disabled = True
        comment.disabled = True
        button_next.disabled = True
        button_skip.disabled = True
        return
    model_file = open(join(model_folder, list1[counter]), 'rb')
    ib1.value = model_file.read()
    h1.value = list1[counter]
    question_file = open(join(missing_folder, list2[counter]), 'rb')
    ib2.value = question_file.read()
    h2.value = list2_title[counter]
    validation_choice.value = None
    comment.value = ''

def on_button_skip_clicked(event):
    global counter
    counter += 1
    model_file = open(join(model_folder, list1[counter]), 'rb')
    ib1.value = model_file.read()
    h1.value = list1[counter]
    question_file = open(join(missing_folder, list2[counter]), 'rb')
    ib2.value = question_file.read()
    h2.value = list2_title[counter]
    validation_choice.value = None
    comment.value = ''

link_download = widgets.HTML(value='Responses.txt')
def on_download_clicked(event):
    global res
    res += list2[counter].split('_')[0] + '\t' + list2[counter].split('_')[1].replace('.png', '') \
    + '\t' + str(validation_choice.value) + '\t' + comment.value + '\n'
    b64 = base64.b64encode(res.encode())
    payload = b64.decode()
    link_download.value = "<a download=responses.txt href=\"data:text/csv;base64," + payload + "\" download>Responses.txt</a>"
    
hbox_layout = widgets.Layout()
hbox_layout.width = '100%'
hbox_layout.justify_content = 'space-around'
image_margin = '0 0 0 0'

boxes = []

model_file = open(join(model_folder, list1[counter]), 'rb')
h1 = widgets.HTML(value=list1[counter])
ib1 = widgets.Image(value=model_file.read())
ib1.layout.object_fit = 'contain'
ib1.layout.margin = image_margin
boxb1 = widgets.Box()
boxb1.layout = green_box_layout
boxb1.children = [ib1]
vb1 = widgets.VBox()
vb1.layout.align_items = 'center'
vb1.children = [h1, boxb1]
boxes.append(vb1)

question_file = open(join(missing_folder, list2[counter]), 'rb')
h2 = widgets.HTML(value=list2_title[counter])
ib2 = widgets.Image(value=question_file.read())
ib2.layout.object_fit = 'contain'
ib2.layout.margin = image_margin
boxb2 = widgets.Box()
boxb2.layout = green_box_layout
boxb2.children = [ib2]
validation_choice = widgets.RadioButtons(
    options=[('Keep the blue elements', 'Keep'), ('Delete the blue elements', 'Delete')],
    value=None,
    layout={'width': 'max-content'}, # If the items' names are long
    disabled=False
)
comment = widgets.Textarea(description='Comment:', placeholder='leave your comment here')
button_next = widgets.Button(description='Save & next', tooltip='Save answer and go to the next question', style={'description_width': 'initial'})
button_next.on_click(on_button_next_clicked)

button_skip = widgets.Button(description='Skip', tooltip='Skip this question and go to the next question', style={'description_width': 'initial'})
button_skip.on_click(on_button_skip_clicked)

next_skip = widgets.HBox()
next_skip.children = [button_next, button_skip]

#button_download = widgets.Button(description='Save responses to file',
#                                 tooltip='Generate a file containing the current response and all of your previous responses',
#                                 style={'description_width': 'initial'})
#button_download.on_click(on_download_clicked)
#download = widgets.HBox()
#download.children = [button_download, link_download]

vb2 = widgets.VBox()
vb2.layout.align_items = 'center'
vb2.children = [h2, boxb2, validation_choice, comment, next_skip, link_download]
boxes.append(vb2)

vb = widgets.VBox()
h = widgets.HTML(value='<{size}>Validation of supposedly missing derivations (in blue)</{size}>'.format(size='h2'))
vb.layout.align_items = 'center'
hb = widgets.HBox()
hb.layout = hbox_layout
hb.children = boxes

In [ ]:
footer = widgets.VBox()
footer_header = widgets.HTML(value='<{size}>Graph notations</{size}>'.format(size='h2'))
footer_content = widgets.HBox()
help_file = open('help.png', 'rb')
help_image = widgets.Image(value=help_file.read(), format='png')
help_image.layout.object_fit = 'contain'
edge_origin = widgets.HTML(value='Edge label: <code>[origin]: [construction1]-[construction2]</code><br /> \
                           <code>origin</code>: \
                           <ul> <li>C = converts</li> <li>D = demonette1</li> <li>Na = denomXaire</li>\
                           <li>Nb = denomXal</li> <li>Nc = denomXique</li> <li>Ma = dimocXaie</li> \
                           <li>Mb = dimocXat</li> <li>Mc = dimocXet</li> <li>Md = dimocXier</li> \
                           <li>Ra = derifantiX</li> <li>Rb = derifde1X</li> <li>Rc = derifenX</li> \
                           <li>Rd = derifinX</li> <li>Re = derifQUANTX</li> <li>Rf = derifreX</li> \
                           <li>Rg = deriftriX</li> <li>Rh = derifXable</li> <li>Ri = derifXiser</li> \
                           </ul> \
                           ')
footer_content.children = [edge_origin, help_image]
footer.children = [footer_header, footer_content]

In [ ]:
vb.children = [h, hb, footer]
display(vb)